<a href="https://colab.research.google.com/github/Ahnkyuwon504/AI-modeling/blob/main/papers_code/torch_knowledge_distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Import Modules

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time

# 2. DataSets

In [2]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 11732119.79it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 351747.02it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 3239609.01it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 4384471.52it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [3]:
print(train_loader.dataset.data.shape, test_loader.dataset.data.shape)

torch.Size([60000, 28, 28]) torch.Size([10000, 28, 28])


# 3. Teacher/Student class Modeling

- forward: `PyTorch의 nn.Module 클래스`에서 상속받은 함수로, 반드시 정의해야 하는 함수
- 입력 데이터 X는 [batch_size, channels, height, width]
- 따라서 x.size(0)는 batch_size=64개의 이미지
- 입력 데이터 X를 1차원 벡터로 변환해 fully connected 레이어에 입력으로 사용

In [4]:
class TeacherModel(nn.Module):
    def __init__(self):
        super(TeacherModel, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(28*28, 400),
            nn.ReLU(),
            nn.Linear(400, 100),
            nn.ReLU(),
            nn.Linear(100, 10)
        )

    def forward(self, x):
        return self.fc(x.view(x.size(0), -1))

class StudentModel(nn.Module):
    def __init__(self):
        super(StudentModel, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(28*28, 100),
            nn.ReLU(),
            nn.Linear(100, 10)
        )

    def forward(self, x):
        return self.fc(x.view(x.size(0), -1))

# 4. 손실함수/옵티마이저

optimizer = optim.Adam(teacher_model.parameters(), lr=0.005)
student_model의 파라미터를 옵티마이저에 지정해서 꽤나 뻘짓 했다.

In [6]:
# 초기화
teacher_model = TeacherModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(teacher_model.parameters(), lr=0.005)

# 5. 학습(Teacher)

In [7]:
num_epochs = 5
teacher_start_time = time.time()  # 평가 시작 시간 기록

for epoch in range(num_epochs):
    teacher_model.train()  # 모델을 학습 모드로 전환
    running_loss = 0.0
    start_time = time.time()  # 평가 시작 시간 기록

    for images, labels in train_loader:  # 데이터로더에서 배치를 가져옴
        outputs = teacher_model(images)  # 모델을 통해 예측값 생성
        loss = criterion(outputs, labels)  # 예측값과 실제 정답을 비교하여 손실 계산

        optimizer.zero_grad()  # 이전 배치의 그래디언트를 초기화
        loss.backward()  # 손실에 대한 그래디언트를 계산 (역전파)
        optimizer.step()  # 계산된 그래디언트를 사용하여 모델 파라미터 업데이트

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

teacher_end_time = time.time()  # 평가 시작 시간 기록
teacher_learning_time = teacher_end_time - teacher_start_time  # 소요 시간 계산
print(f'Learning Time: {teacher_learning_time:.4f} seconds')

Epoch [1/5], Loss: 0.3454
Epoch [2/5], Loss: 0.1805
Epoch [3/5], Loss: 0.1563
Epoch [4/5], Loss: 0.1354
Epoch [5/5], Loss: 0.1309
Learning Time: 94.4465 seconds


> 여기까지는 일반적인 심층신경망 학습과 동일

# 6. Soft Targets 추출

eval()
- Dropout 및 Batch Normalization과 같은 레이어가 학습 시와 다르게 동작하도록 설정
- 학습 모드에서는 Dropout이 활성화되어 일부 뉴런을 무작위로 꺼버리지만, 평가 모드에서는 모든 뉴런이 활성화

torch.no_grad()
- 그래디언트 계산 비활성화
- 그래디언트가 계산되지 않으므로 메모리 사용량이 줄고 속도 향상
- 모델 평가/예측 시에 주로 사용

In [8]:
def get_soft_targets(model, dataloader):
    model.eval()  # 모델을 평가 모드로 전환
    soft_targets = []  # 소프트 타겟을 저장할 리스트 초기화
    with torch.no_grad():  # 그래디언트 계산을 비활성화 (메모리 절약 및 속도 향상)
        for images, _ in dataloader:  # 데이터로더에서 배치를 반복적으로 가져옴
            outputs = model(images)  # 모델을 통해 예측값 생성
            soft_targets.append(outputs)  # 예측값을 소프트 타겟 리스트에 추가
    return torch.cat(soft_targets)  # 모든 배치의 예측값을 하나의 텐서로 결합하여 반환

soft_targets = get_soft_targets(teacher_model, train_loader)

# 7. 학습(Student)

Hard Loss
- Student 모델의 출력값과 실제 레이블 간의 손실

Soft Loss
- Student 모델의 출력값과 Teacher 모델의 soft targets 간의 손실을 계산
- KL Divergence 손실을 사용하여 두 확률 분포 간의 차이를 측정

Temperature
- Temperature는 Softmax 함수의 출력을 부드럽게 만드는 역할
- T가 높을수록 확률 분포가 더 부드러워지고,이는 Teacher 모델의 soft targets를 부드럽게 만들어 Student 모델이 더 잘 학습할 수 있도록 합니다.

Alpha
- Hard loss와 Soft loss 간의 가중치를 조절하는 하이퍼파라미터
- Alpha가 0에 가까울수록 Soft loss가 더 많이 반영되고, 1에 가까울수록 Hard loss가 더 많이 반영.

nn.KLDivLoss()
- 두 확률 분포 간의 차이를 계산하는 손실 함수

KL Divergence(Kullback-Leibler Divergence)
- 두 확률 분포 P와 Q 간의 차이를 측정하는 비대칭적인 측도


In [9]:
student_model = StudentModel()
optimizer = optim.Adam(student_model.parameters(), lr=0.005)

# Student Model 학습
def distillation_loss(outputs, targets, soft_targets, T, alpha):
    hard_loss = criterion(outputs, targets)
    soft_loss = nn.KLDivLoss()(nn.functional.log_softmax(outputs/T, dim=1), nn.functional.softmax(soft_targets/T, dim=1))
    return alpha * hard_loss + (1 - alpha) * soft_loss

alpha = 0.5
T = 2.0

student_start_time = time.time()  # 평가 시작 시간 기록

student_model.train()
for epoch in range(5):
    for i, (images, labels) in enumerate(train_loader):
        outputs = student_model(images)
        loss = distillation_loss(outputs, labels, soft_targets[i*64:(i+1)*64], T, alpha)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/5], Loss: {loss.item():.4f}')

student_end_time = time.time()  # 평가 시작 시간 기록
student_learning_time = student_end_time - student_start_time  # 소요 시간 계산
print(f'Learning Time: {student_learning_time:.4f} seconds')

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:2976: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


Epoch [1/5], Loss: 0.3004
Epoch [2/5], Loss: 0.2195
Epoch [3/5], Loss: 0.1888
Epoch [4/5], Loss: 0.2291
Epoch [5/5], Loss: 0.2002
Learning Time: 72.3704 seconds


In [10]:
# 모델 평가 함수
def evaluate_model(model, test_loader):
    model.eval()  # 평가 모드로 전환
    correct = 0
    total = 0
    start_time = time.time()  # 평가 시작 시간 기록

    with torch.no_grad():  # 그래디언트 계산 비활성화
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    end_time = time.time()  # 평가 종료 시간 기록
    accuracy = correct / total
    evaluation_time = end_time - start_time  # 소요 시간 계산
    return accuracy, evaluation_time

# Teacher 모델 성능 평가
teacher_accuracy, teacher_time = evaluate_model(teacher_model, test_loader)
print('# Teacher')
print(f'Learning Time: {teacher_learning_time:.4f} seconds')
print(f'Test Accuracy: {teacher_accuracy * 100:.2f}%')
print(f'Inference Time: {teacher_time:.4f} seconds')

# Student 모델 성능 평가
student_accuracy, student_time = evaluate_model(student_model, test_loader)
print('# Student')
print(f'Learning Time: {student_learning_time:.4f} seconds')
print(f'Test Accuracy: {student_accuracy * 100:.2f}%')
print(f'Inference Time: {student_time:.4f} seconds')


# Teacher
Learning Time: 94.4465 seconds
Test Accuracy: 95.85%
Inference Time: 2.0006 seconds
# Student
Learning Time: 72.3704 seconds
Test Accuracy: 96.13%
Inference Time: 1.9377 seconds
